In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stockdata.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 17




#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 17





# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [3]:
#we store the variables here
log_files_path = 'C:/Users/jira_/Desktop/new_st22-17-22/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [22, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 22], [22], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '17'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 22]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/new_st22-17-22/tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,22)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.416327397
Validation Loss: 3.5520735
Epoch: 0002 cost = 2.397097071
Validation Loss: 2.8889966
Epoch: 0003 cost = 2.377463539
Validation Loss: 2.4703097
Epoch: 0004 cost = 2.357883652
Validation Loss: 2.2672603
Epoch: 0005 cost = 2.338691314
Validation Loss: 2.1903665
Epoch: 0006 cost = 2.319817901
Validation Loss: 2.1434493
Epoch: 0007 cost = 2.301469207
Validation Loss: 2.1068149
Epoch: 0008 cost = 2.283548594
Validation Loss: 2.0744815
Epoch: 0009 cost = 2.265431245
Validation Loss: 2.0318727
Epoch: 0010 cost = 2.246607900
Validation Loss: 2.0106578
Epoch: 0011 cost = 2.226903081
Validation Loss: 1.9854156
Epoch: 0012 cost = 2.206109325
Validation Loss: 1.9538109
Epoch: 0013 cost = 2.184526404
Validation Loss: 1.9261515
Epoch: 0014 cost = 2.164566914
Validation Loss: 1.9198514
Epoch: 0015 cost = 2.147708257
Validation Loss: 1.95495
Epoch: 0016 cost = 2.133254051
Validation Loss: 1.9930886
Epoch: 0017 cost = 2.120469610
Validation Loss: 2.002666
Epoch: 0

Epoch: 0154 cost = 1.168863078
Validation Loss: 1.1006485
Epoch: 0155 cost = 1.164080898
Validation Loss: 1.075369
Epoch: 0156 cost = 1.159319222
Validation Loss: 1.0510638
Epoch: 0157 cost = 1.154578487
Validation Loss: 1.0737696
Epoch: 0158 cost = 1.149858435
Validation Loss: 1.073888
Epoch: 0159 cost = 1.145158847
Validation Loss: 1.0741583
Epoch: 0160 cost = 1.140479664
Validation Loss: 1.0835078
Epoch: 0161 cost = 1.135820846
Validation Loss: 1.099482
Epoch: 0162 cost = 1.131182134
Validation Loss: 1.1256515
Epoch: 0163 cost = 1.126563648
Validation Loss: 1.137619
Epoch: 0164 cost = 1.121964792
Validation Loss: 1.1417971
Epoch: 0165 cost = 1.117386301
Validation Loss: 1.0865923
Epoch: 0166 cost = 1.112827082
Validation Loss: 1.0562782
Epoch: 0167 cost = 1.108287752
Validation Loss: 1.0110555
Epoch: 0168 cost = 1.103767991
Validation Loss: 0.7874173
Epoch: 0169 cost = 1.099267443
Validation Loss: 0.8458731
Epoch: 0170 cost = 1.094786604
Validation Loss: 0.96003014
Epoch: 0171 cost 

Epoch: 0299 cost = 0.661985795
Validation Loss: 0.60986966
Epoch: 0300 cost = 0.659635812
Validation Loss: 0.61687446
Epoch: 0301 cost = 0.657298485
Validation Loss: 0.62275344
Epoch: 0302 cost = 0.654973765
Validation Loss: 0.6302465
Epoch: 0303 cost = 0.652661403
Validation Loss: 0.6445244
Epoch: 0304 cost = 0.650361498
Validation Loss: 0.66754127
Epoch: 0305 cost = 0.648073753
Validation Loss: 0.6718888
Epoch: 0306 cost = 0.645798266
Validation Loss: 0.6869734
Epoch: 0307 cost = 0.643534889
Validation Loss: 0.6852595
Epoch: 0308 cost = 0.641283582
Validation Loss: 0.6874002
Epoch: 0309 cost = 0.639044464
Validation Loss: 0.687982
Epoch: 0310 cost = 0.636817068
Validation Loss: 0.6974135
Epoch: 0311 cost = 0.634601553
Validation Loss: 0.6948051
Epoch: 0312 cost = 0.632397910
Validation Loss: 0.6719158
Epoch: 0313 cost = 0.630205959
Validation Loss: 0.6563579
Epoch: 0314 cost = 0.628025691
Validation Loss: 0.6448611
Epoch: 0315 cost = 0.625857184
Validation Loss: 0.6442804
Epoch: 0316

Epoch: 0440 cost = 0.426883931
Validation Loss: 0.4170485
Epoch: 0441 cost = 0.425754224
Validation Loss: 0.40512526
Epoch: 0442 cost = 0.424630190
Validation Loss: 0.43263277
Epoch: 0443 cost = 0.423511619
Validation Loss: 0.4465251
Epoch: 0444 cost = 0.422398483
Validation Loss: 0.4606495
Epoch: 0445 cost = 0.421290542
Validation Loss: 0.4498432
Epoch: 0446 cost = 0.420188099
Validation Loss: 0.44802845
Epoch: 0447 cost = 0.419090743
Validation Loss: 0.43834126
Epoch: 0448 cost = 0.417998676
Validation Loss: 0.4340988
Epoch: 0449 cost = 0.416911726
Validation Loss: 0.4224449
Epoch: 0450 cost = 0.415829718
Validation Loss: 0.43040058
Epoch: 0451 cost = 0.414752662
Validation Loss: 0.4115979
Epoch: 0452 cost = 0.413680588
Validation Loss: 0.4133329
Epoch: 0453 cost = 0.412613094
Validation Loss: 0.41446966
Epoch: 0454 cost = 0.411550308
Validation Loss: 0.41673768
Epoch: 0455 cost = 0.410492003
Validation Loss: 0.40799117
Epoch: 0456 cost = 0.409438188
Validation Loss: 0.4071988
Epoch:

Epoch: 0587 cost = 0.298618759
Validation Loss: 0.31511012
Epoch: 0588 cost = 0.298029721
Validation Loss: 0.30938044
Epoch: 0589 cost = 0.297443425
Validation Loss: 0.3149911
Epoch: 0590 cost = 0.296859761
Validation Loss: 0.3186989
Epoch: 0591 cost = 0.296278904
Validation Loss: 0.32512742
Epoch: 0592 cost = 0.295700635
Validation Loss: 0.32955572
Epoch: 0593 cost = 0.295124958
Validation Loss: 0.32382303
Epoch: 0594 cost = 0.294551929
Validation Loss: 0.3248861
Epoch: 0595 cost = 0.293981671
Validation Loss: 0.30902162
Epoch: 0596 cost = 0.293413982
Validation Loss: 0.28933054
Epoch: 0597 cost = 0.292848955
Validation Loss: 0.30500212
Epoch: 0598 cost = 0.292286322
Validation Loss: 0.30804172
Epoch: 0599 cost = 0.291726326
Validation Loss: 0.31060943
Epoch: 0600 cost = 0.291169018
Validation Loss: 0.3010936
Epoch: 0601 cost = 0.290614183
Validation Loss: 0.29201958
Epoch: 0602 cost = 0.290061990
Validation Loss: 0.2887677
Epoch: 0603 cost = 0.289512242
Validation Loss: 0.3025019
Epo

...

Epoch: 4408 cost = 0.027490086
Validation Loss: 0.037805706
Epoch: 4409 cost = 0.027497534
Validation Loss: 0.03248333
Epoch: 4410 cost = 0.027493794
Validation Loss: 0.041124042
Epoch: 4411 cost = 0.027466048
Validation Loss: 0.045455016
Epoch: 4412 cost = 0.027509206
Validation Loss: 0.041170757
Epoch: 4413 cost = 0.027481678
Validation Loss: 0.060093917
Epoch: 4414 cost = 0.027479666
Validation Loss: 0.1468629
Epoch: 4415 cost = 0.027466292
Validation Loss: 0.09474124
Epoch: 4416 cost = 0.027514369
Validation Loss: 0.07686862
Epoch: 4417 cost = 0.027464840
Validation Loss: 0.058847502
Epoch: 4418 cost = 0.027481924
Validation Loss: 0.04340712
Epoch: 4419 cost = 0.027490204
Validation Loss: 0.033723082
Epoch: 4420 cost = 0.027484347
Validation Loss: 0.049685024
Epoch: 4421 cost = 0.027458396
Validation Loss: 0.042994168
Epoch: 4422 cost = 0.027503721
Validation Loss: 0.035496444
Epoch: 4423 cost = 0.027471036
Validation Loss: 0.032104112
Epoch: 4424 cost = 0.027470452
Validation Loss

Epoch: 4553 cost = 0.027389280
Validation Loss: 0.055165328
Epoch: 4554 cost = 0.027359341
Validation Loss: 0.043296054
Epoch: 4555 cost = 0.027425147
Validation Loss: 0.048007768
Epoch: 4556 cost = 0.027362784
Validation Loss: 0.09890389
Epoch: 4557 cost = 0.027389890
Validation Loss: 0.124783635
Epoch: 4558 cost = 0.027384230
Validation Loss: 0.096550055
Epoch: 4559 cost = 0.027399759
Validation Loss: 0.080800176
Epoch: 4560 cost = 0.027349484
Validation Loss: 0.045675207
Epoch: 4561 cost = 0.027395862
Validation Loss: 0.037400935
Epoch: 4562 cost = 0.027378513
Validation Loss: 0.043592345
Epoch: 4563 cost = 0.027385291
Validation Loss: 0.039247382
Epoch: 4564 cost = 0.027354862
Validation Loss: 0.037907187
Epoch: 4565 cost = 0.027421264
Validation Loss: 0.032292735
Epoch: 4566 cost = 0.027354668
Validation Loss: 0.05259503
Epoch: 4567 cost = 0.027383575
Validation Loss: 0.06488397
Epoch: 4568 cost = 0.027379132
Validation Loss: 0.060138512
Epoch: 4569 cost = 0.027390732
Validation L

Epoch: 4699 cost = 0.027306803
Validation Loss: 0.03508571
Epoch: 4700 cost = 0.027267935
Validation Loss: 0.036415394
Epoch: 4701 cost = 0.027342325
Validation Loss: 0.037059143
Epoch: 4702 cost = 0.027269440
Validation Loss: 0.038358368
Epoch: 4703 cost = 0.027303336
Validation Loss: 0.049822427
Epoch: 4704 cost = 0.027288777
Validation Loss: 0.04577042
Epoch: 4705 cost = 0.027314077
Validation Loss: 0.04704235
Epoch: 4706 cost = 0.027255753
Validation Loss: 0.037361488
Epoch: 4707 cost = 0.027305036
Validation Loss: 0.044271305
Epoch: 4708 cost = 0.027290329
Validation Loss: 0.05456164
Epoch: 4709 cost = 0.027297405
Validation Loss: 0.07427678
Epoch: 4710 cost = 0.027261732
Validation Loss: 0.06551751
Epoch: 4711 cost = 0.027335165
Validation Loss: 0.05745615
Epoch: 4712 cost = 0.027263581
Validation Loss: 0.049233522
Epoch: 4713 cost = 0.027301053
Validation Loss: 0.04558994
Epoch: 4714 cost = 0.027287087
Validation Loss: 0.037213486
Epoch: 4715 cost = 0.027307569
Validation Loss: 

Epoch: 4845 cost = 0.027220042
Validation Loss: 0.038422767
Epoch: 4846 cost = 0.027182629
Validation Loss: 0.03943801
Epoch: 4847 cost = 0.027257913
Validation Loss: 0.034445703
Epoch: 4848 cost = 0.027178786
Validation Loss: 0.036827475
Epoch: 4849 cost = 0.027218751
Validation Loss: 0.047955923
Epoch: 4850 cost = 0.027206872
Validation Loss: 0.04555257
Epoch: 4851 cost = 0.027224215
Validation Loss: 0.093987845
Epoch: 4852 cost = 0.027170661
Validation Loss: 0.07072533
Epoch: 4853 cost = 0.027231597
Validation Loss: 0.05406325
Epoch: 4854 cost = 0.027197423
Validation Loss: 0.052560043
Epoch: 4855 cost = 0.027212638
Validation Loss: 0.03292417
Epoch: 4856 cost = 0.027178285
Validation Loss: 0.033054445
Epoch: 4857 cost = 0.027250912
Validation Loss: 0.03159526
Epoch: 4858 cost = 0.027170757
Validation Loss: 0.041265074
Epoch: 4859 cost = 0.027211039
Validation Loss: 0.04104137
Epoch: 4860 cost = 0.027202560
Validation Loss: 0.039658442
Epoch: 4861 cost = 0.027215288
Validation Loss:

Epoch: 4988 cost = 0.027094363
Validation Loss: 0.03439636
Epoch: 4989 cost = 0.027151454
Validation Loss: 0.056484874
Epoch: 4990 cost = 0.027123905
Validation Loss: 0.05064976
Epoch: 4991 cost = 0.027144915
Validation Loss: 0.05284453
Epoch: 4992 cost = 0.027103153
Validation Loss: 0.036297914
Epoch: 4993 cost = 0.027175115
Validation Loss: 0.03692411
Epoch: 4994 cost = 0.027099599
Validation Loss: 0.037163533
Epoch: 4995 cost = 0.027143190
Validation Loss: 0.036941797
Epoch: 4996 cost = 0.027125139
Validation Loss: 0.06102747
Epoch: 4997 cost = 0.027149998
Validation Loss: 0.05777337
Epoch: 4998 cost = 0.027089588
Validation Loss: 0.035202418
Epoch: 4999 cost = 0.027148540
Validation Loss: 0.033088394
Epoch: 5000 cost = 0.027118154
Validation Loss: 0.040828925
Optimization Finished!
Test Loss: 0.038509537


In [5]:
import csv
with open('C:/Users/jira_/Desktop/new_stresult22-17-22.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))